### Installations and Importing

In [2]:
!pip install -q transformers accelerate bert-score

from transformers import BlipProcessor, BlipForQuestionAnswering
from PIL import Image
import torch
import pandas as pd
from tqdm import tqdm
from bert_score import score
import os


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.6 MB/s eta 0:00:00:00:0100:01


2025-05-16 13:41:55.245303: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747402915.445335      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747402915.500830      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Load processor and model

In [3]:

processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base").to("cuda")
model.eval()


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

BlipForQuestionAnswering(
  (vision_model): BlipVisionModel(
    (embeddings): BlipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (encoder): BlipEncoder(
      (layers): ModuleList(
        (0-11): 12 x BlipEncoderLayer(
          (self_attn): BlipAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (projection): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): BlipMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((768,), eps=1e-05, e

# Load dataset (20000,5000, 500 sample for comparison)

In [ ]:
df = pd.read_csv("/kaggle/input/dataforvr/complete_updated.csv")  
df = df.dropna(subset=["path", "question", "answer"])
df_5k = df.sample(n=5000, random_state=42).reset_index(drop=True)
df_500=df.sample(n=500,random_state=42).reset_index(drop=True)
df_20k=df.sample(n=20000,random_state=42).reset_index(drop=True)

##  Inference Function

In [ ]:
def get_prediction(image_path, question):
    image = Image.open(image_path).convert('RGB')
    inputs = processor(image, question, return_tensors="pt").to("cuda", torch.float16)
    output = model.generate(**inputs, max_new_tokens=5)
    pred = processor.decode(output[0], skip_special_tokens=True)
    return pred.strip().lower().split()[0] 

## Inference Loop

In [ ]:
def run_vqa(df_input, save_path):
    results = []
    for _, row in tqdm(df_input.iterrows(), total=len(df_input)):
        try:
            image_path = "/kaggle/input/abo-small/vr_project/abo-images-small/images/small/" + row["path"]
            question = row["question"]
            ground_truth = row["answer"].strip().lower()
            prediction = get_prediction(image_path, question)

            is_correct = (prediction == ground_truth)
            token_f1 = 1.0 if is_correct else 0.0

            results.append({
                "image_id": row["image_id"],
                "image": row["path"],
                "question": question,
                "ground_truth": ground_truth,
                "prediction": prediction,
                "correct": is_correct,
                "token_f1": token_f1
            })

        except Exception as e:
            results.append({
                "image_id": row["image_id"],
                "image": row["path"],
                "question": row["question"],
                "ground_truth": row["answer"],
                "prediction": "error",
                "correct": False,
                "token_f1": 0.0
            })

    results_df = pd.DataFrame(results)


    P, R, F1 = score(results_df["prediction"].tolist(), results_df["ground_truth"].tolist(), lang="en", verbose=True)
    results_df["bertscore_f1"] = F1.tolist()


    results_df.to_csv(save_path, index=False)
    return results_df


## Run on all sets

In [7]:
results_5k = run_vqa(df_5k, "vqa_5k_baseline.csv")
results_500 = run_vqa(df_500, "vqa_500_baseline.csv")
results_20k = run_vqa(df_20k, "vqa_20k_baseline.csv")

100%|██████████| 5000/5000 [07:07<00:00, 11.71it/s]


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/52 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/313 [00:00<?, ?it/s]

done in 5.29 seconds, 3783.26 sentences/sec


In [8]:
def summarize(df, label):
    print(f"--- {label} ---")
    acc = df["correct"].mean()
    f1_token = df["token_f1"].mean()
    f1_bert = df["bertscore_f1"].mean()
    print(f"Accuracy:     {acc:.4f}")
    print(f"Token-F1:     {f1_token:.4f}")
    print(f"BERTScore-F1: {f1_bert:.4f}")

summarize(results_5k, "Full 5k")
summarize(results_500, "Subsample 500")
summarize(results_20k, "Full 20k")


--- Full 5k ---
Accuracy:     0.3496
Token-F1:     0.3496
BERTScore-F1: 0.9592
--- Subsample 500 ---
Accuracy:     0.3260
Token-F1:     0.3260
BERTScore-F1: 0.9596
--- Full 20k ---
Accuracy:     0.3525
Token-F1:     0.3525
BERTScore-F1: 0.9611
